In [1]:
import math
import random

import numpy as np
import matplotlib.pyplot as plt

import squigglepy as sq
from squigglepy import bayes
from squigglepy.numbers import K, M, B, T

from copy import copy, deepcopy
from scipy import stats
from pprint import pprint
from tqdm import tqdm
from datetime import datetime as dt
print('Loaded 1')

exec(open('utils.py').read())
print('Loaded 2')

exec(open('modules/tai_timelines.py').read())
print('Loaded TAI timelines module')

exec(open('modules/anchors.py').read())
print('Loaded anchors module')

exec(open('/Users/peterwildeford/dev/forecastflow/library.py').read()) # TODO: Package?
print('Loaded Metaculus module')

Loaded 1
Loaded 2
Loaded TAI timelines module
Loaded anchors module
Loaded Metaculus lib v0.3
Loaded Metaculus module


## AI Timelines Sensitivity Analysis

In [2]:
CURRENT_YEAR = 2024                               # What year to start the run on? (default: 2023)
MAX_YEAR = 2124                                   # What year to end the run on? (default: 2123)
years = list(range(CURRENT_YEAR, MAX_YEAR))

def print_year(y):
    y = y['tai_year']
    return '>{}'.format(MAX_YEAR) if y > MAX_YEAR else str(int(y))


defaults = {'initial_gdp_': 23*T,
            'tai_flop_size_': 30,
            'algo_doubling_rate_': algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=30),
            'possible_algo_reduction_': possible_algo_reduction_fn(min_reduction=2,
                                                                   max_reduction=7,
                                                                   tai_flop_size=30),
            'initial_flop_per_dollar_': 10 ** 18.4,
            'flop_halving_rate_': 2.54,
            'max_flop_per_dollar_': 10 ** 24,
            'initial_pay_': 10 ** 8.7,
            'gdp_growth_': 1.02,
            'max_gdp_frac_': 0.0004,
            'willingness_ramp_': 1,
            'spend_doubling_time_': 3,
            'nonscaling_delay_': {'delay': {'prob': 1, 'length': 4}},
            'willingness_spend_horizon_': 1,
            'print_diagnostic': False,
            'variables': {'CURRENT_YEAR': CURRENT_YEAR, 'MAX_YEAR': MAX_YEAR}}

print('## Default ##')
result = run_tai_model_round(**defaults)
print('{}: {}'.format('Default', print_year(result)))          

## Default ##
Default: 2039


In [3]:
print('## TAI FLOP Size ##')
for t in range(23, 46):
    varset = deepcopy(defaults)
    varset['tai_flop_size_'] = t
    varset['algo_doubling_rate_'] = algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=t)
    varset['possible_algo_reduction_'] = possible_algo_reduction_fn(min_reduction=2,
                                                                    max_reduction=5,
                                                                    tai_flop_size=t)
    result = run_tai_model_round(**varset)
    print('FLOP Size for TAI {} log FLOP -> {}'.format(t, print_year(result)))

## TAI FLOP Size ##
FLOP Size for TAI 23 log FLOP -> 2028
FLOP Size for TAI 24 log FLOP -> 2028
FLOP Size for TAI 25 log FLOP -> 2028
FLOP Size for TAI 26 log FLOP -> 2028
FLOP Size for TAI 27 log FLOP -> 2028
FLOP Size for TAI 28 log FLOP -> 2032
FLOP Size for TAI 29 log FLOP -> 2035
FLOP Size for TAI 30 log FLOP -> 2039
FLOP Size for TAI 31 log FLOP -> 2042
FLOP Size for TAI 32 log FLOP -> 2045
FLOP Size for TAI 33 log FLOP -> 2051
FLOP Size for TAI 34 log FLOP -> 2057
FLOP Size for TAI 35 log FLOP -> 2057
FLOP Size for TAI 36 log FLOP -> 2065
FLOP Size for TAI 37 log FLOP -> 2075
FLOP Size for TAI 38 log FLOP -> 2075
FLOP Size for TAI 39 log FLOP -> >2124
FLOP Size for TAI 40 log FLOP -> >2124
FLOP Size for TAI 41 log FLOP -> >2124
FLOP Size for TAI 42 log FLOP -> >2124
FLOP Size for TAI 43 log FLOP -> >2124
FLOP Size for TAI 44 log FLOP -> >2124
FLOP Size for TAI 45 log FLOP -> >2124


In [4]:
print('## GDP Growth ##')
for g in range(0, 5):
    varset = deepcopy(defaults)
    varset['gdp_growth_'] = 1 + (g / 100)
    result = run_tai_model_round(**varset)
    print('GDP Growth Rate {}% -> {}'.format(1 + g / 100, print_year(result)))

## GDP Growth ##
GDP Growth Rate 1.0% -> 2039
GDP Growth Rate 1.01% -> 2039
GDP Growth Rate 1.02% -> 2039
GDP Growth Rate 1.03% -> 2039
GDP Growth Rate 1.04% -> 2039


In [5]:
print('## Max GDP Frac ##')
for g in [1/(500*K), 1/(250*K), 1/(100*K), 1/(50*K), 1/(10*K), 5/(10*K),
          1/2000, 1/1500, 1/1000, 2/1000, 3/1000, 4/1000, 5/1000, 1/100,
          2/100, 3/100, 4/100]:
    varset = deepcopy(defaults)
    varset['max_gdp_frac_'] = g
    result = run_tai_model_round(**varset)
    print('Max GDP Frac {} (1 in ~{:,}) -> {}'.format(round(g, 6), int(round(1 / g)), print_year(result)))

## Max GDP Frac ##
Max GDP Frac 2e-06 (1 in ~500,000) -> 2048
Max GDP Frac 4e-06 (1 in ~250,000) -> 2046
Max GDP Frac 1e-05 (1 in ~100,000) -> 2044
Max GDP Frac 2e-05 (1 in ~50,000) -> 2043
Max GDP Frac 0.0001 (1 in ~10,000) -> 2040
Max GDP Frac 0.0005 (1 in ~2,000) -> 2039
Max GDP Frac 0.0005 (1 in ~2,000) -> 2039
Max GDP Frac 0.000667 (1 in ~1,500) -> 2038
Max GDP Frac 0.001 (1 in ~1,000) -> 2038
Max GDP Frac 0.002 (1 in ~500) -> 2038
Max GDP Frac 0.003 (1 in ~333) -> 2038
Max GDP Frac 0.004 (1 in ~250) -> 2038
Max GDP Frac 0.005 (1 in ~200) -> 2038
Max GDP Frac 0.01 (1 in ~100) -> 2038
Max GDP Frac 0.02 (1 in ~50) -> 2038
Max GDP Frac 0.03 (1 in ~33) -> 2038
Max GDP Frac 0.04 (1 in ~25) -> 2038


In [6]:
print('## Spend Doubling Time ##')
for d in range(0, 101, 2):
    varset = deepcopy(defaults)
    varset['spend_doubling_time_'] = 1 + (d / 10)
    result = run_tai_model_round(**varset)
    print('Spend Doubling Time {}yrs -> {}'.format(1 + d / 10, print_year(result)))

## Spend Doubling Time ##
Spend Doubling Time 1.0yrs -> 2036
Spend Doubling Time 1.2yrs -> 2037
Spend Doubling Time 1.4yrs -> 2037
Spend Doubling Time 1.6yrs -> 2037
Spend Doubling Time 1.8yrs -> 2037
Spend Doubling Time 2.0yrs -> 2038
Spend Doubling Time 2.2yrs -> 2038
Spend Doubling Time 2.4yrs -> 2038
Spend Doubling Time 2.6yrs -> 2038
Spend Doubling Time 2.8yrs -> 2039
Spend Doubling Time 3.0yrs -> 2039
Spend Doubling Time 3.2yrs -> 2039
Spend Doubling Time 3.4yrs -> 2039
Spend Doubling Time 3.6yrs -> 2039
Spend Doubling Time 3.8yrs -> 2039
Spend Doubling Time 4.0yrs -> 2039
Spend Doubling Time 4.2yrs -> 2040
Spend Doubling Time 4.4yrs -> 2040
Spend Doubling Time 4.6yrs -> 2040
Spend Doubling Time 4.8yrs -> 2040
Spend Doubling Time 5.0yrs -> 2040
Spend Doubling Time 5.2yrs -> 2040
Spend Doubling Time 5.4yrs -> 2040
Spend Doubling Time 5.6yrs -> 2040
Spend Doubling Time 5.8yrs -> 2040
Spend Doubling Time 6.0yrs -> 2040
Spend Doubling Time 6.2yrs -> 2040
Spend Doubling Time 6.4yrs ->

In [7]:
print('## Initial FLOP per dollar ##')
for d in [17, 17.5, 18, 18.3, 18.5, 19]:
    varset = deepcopy(defaults)
    varset['initial_flop_per_dollar_'] = 10 ** d
    result = run_tai_model_round(**varset)
    print('Initial log FLOP per dollar {} -> {}'.format(d, print_year(result)))
    

## Initial FLOP per dollar ##
Initial log FLOP per dollar 17 -> 2044
Initial log FLOP per dollar 17.5 -> 2042
Initial log FLOP per dollar 18 -> 2040
Initial log FLOP per dollar 18.3 -> 2039
Initial log FLOP per dollar 18.5 -> 2038
Initial log FLOP per dollar 19 -> 2037


In [8]:
print('## Initial pay ##')
for p in range(70, 101, 2):
    varset = deepcopy(defaults)
    varset['initial_pay_'] = 10 ** (p / 10)
    result = run_tai_model_round(**varset)
    print('Initial pay in log 2022$USD {} (~${}) -> {}'.format(p / 10, numerize(10 ** (p / 10)), print_year(result)))

## Initial pay ##
Initial pay in log 2022$USD 7.0 (~$10.0 million) -> 2044
Initial pay in log 2022$USD 7.2 (~$15.8 million) -> 2043
Initial pay in log 2022$USD 7.4 (~$25.1 million) -> 2043
Initial pay in log 2022$USD 7.6 (~$39.8 million) -> 2042
Initial pay in log 2022$USD 7.8 (~$63.1 million) -> 2041
Initial pay in log 2022$USD 8.0 (~$100.0 million) -> 2041
Initial pay in log 2022$USD 8.2 (~$158.5 million) -> 2040
Initial pay in log 2022$USD 8.4 (~$251.2 million) -> 2039
Initial pay in log 2022$USD 8.6 (~$398.1 million) -> 2039
Initial pay in log 2022$USD 8.8 (~$631.0 million) -> 2038
Initial pay in log 2022$USD 9.0 (~$1.0 billion) -> 2038
Initial pay in log 2022$USD 9.2 (~$1.6 billion) -> 2038
Initial pay in log 2022$USD 9.4 (~$2.5 billion) -> 2037
Initial pay in log 2022$USD 9.6 (~$4.0 billion) -> 2037
Initial pay in log 2022$USD 9.8 (~$6.3 billion) -> 2037
Initial pay in log 2022$USD 10.0 (~$10.0 billion) -> 2037


In [9]:
print('## FLOP halving rate ##')
for f in range(10, 51):
    varset = deepcopy(defaults)
    varset['flop_halving_rate_'] = f / 10
    result = run_tai_model_round(**varset)
    print('FLOP halving rate {} -> {}'.format(f / 10, print_year(result)))

## FLOP halving rate ##
FLOP halving rate 1.0 -> 2035
FLOP halving rate 1.1 -> 2035
FLOP halving rate 1.2 -> 2035
FLOP halving rate 1.3 -> 2036
FLOP halving rate 1.4 -> 2036
FLOP halving rate 1.5 -> 2036
FLOP halving rate 1.6 -> 2037
FLOP halving rate 1.7 -> 2037
FLOP halving rate 1.8 -> 2037
FLOP halving rate 1.9 -> 2037
FLOP halving rate 2.0 -> 2038
FLOP halving rate 2.1 -> 2038
FLOP halving rate 2.2 -> 2038
FLOP halving rate 2.3 -> 2038
FLOP halving rate 2.4 -> 2038
FLOP halving rate 2.5 -> 2039
FLOP halving rate 2.6 -> 2039
FLOP halving rate 2.7 -> 2039
FLOP halving rate 2.8 -> 2039
FLOP halving rate 2.9 -> 2039
FLOP halving rate 3.0 -> 2039
FLOP halving rate 3.1 -> 2040
FLOP halving rate 3.2 -> 2040
FLOP halving rate 3.3 -> 2040
FLOP halving rate 3.4 -> 2040
FLOP halving rate 3.5 -> 2040
FLOP halving rate 3.6 -> 2040
FLOP halving rate 3.7 -> 2040
FLOP halving rate 3.8 -> 2040
FLOP halving rate 3.9 -> 2041
FLOP halving rate 4.0 -> 2041
FLOP halving rate 4.1 -> 2041
FLOP halving rat

In [10]:
print('## Max FLOP per dollar ##')
for f in range(20, 31):
    varset = deepcopy(defaults)
    varset['max_flop_per_dollar_'] = 10 ** f
    result = run_tai_model_round(**varset)
    print('Max log FLOP per 2022USD$1 = {} -> {}'.format(f, print_year(result)))

## Max FLOP per dollar ##
Max log FLOP per 2022USD$1 = 20 -> 2039
Max log FLOP per 2022USD$1 = 21 -> 2039
Max log FLOP per 2022USD$1 = 22 -> 2039
Max log FLOP per 2022USD$1 = 23 -> 2039
Max log FLOP per 2022USD$1 = 24 -> 2039
Max log FLOP per 2022USD$1 = 25 -> 2039
Max log FLOP per 2022USD$1 = 26 -> 2039
Max log FLOP per 2022USD$1 = 27 -> 2039
Max log FLOP per 2022USD$1 = 28 -> 2039
Max log FLOP per 2022USD$1 = 29 -> 2039
Max log FLOP per 2022USD$1 = 30 -> 2039


In [11]:
print('## Algo Doubling Rate Minimum ##')
for m in [1, 1.3, 1.5, 2, 2.5, 3, 3.5]:
    varset = deepcopy(defaults)
    varset['algo_doubling_rate_'] = algo_halving_fn(min_speed=m, max_speed=3.5, tai_flop_size=defaults['tai_flop_size_'])
    result = run_tai_model_round(**varset)
    print('Algo doubling rate minimum {} -> {}'.format(m, print_year(result)))

## Algo Doubling Rate Minimum ##
Algo doubling rate minimum 1 -> 2039
Algo doubling rate minimum 1.3 -> 2039
Algo doubling rate minimum 1.5 -> 2039
Algo doubling rate minimum 2 -> 2039
Algo doubling rate minimum 2.5 -> 2039
Algo doubling rate minimum 3 -> 2039
Algo doubling rate minimum 3.5 -> 2039


In [12]:
print('## Algo Doubling Rate Maximum ##')
for m in [2, 2.5, 3, 3.5, 4, 4.5, 5, 6]:
    varset = deepcopy(defaults)
    varset['algo_doubling_rate_'] = algo_halving_fn(min_speed=2, max_speed=m, tai_flop_size=defaults['tai_flop_size_'])
    result = run_tai_model_round(**varset)
    print('Algo doubling rate maximum {} -> {}'.format(m, print_year(result)))

## Algo Doubling Rate Maximum ##
Algo doubling rate maximum 2 -> 2037
Algo doubling rate maximum 2.5 -> 2038
Algo doubling rate maximum 3 -> 2038
Algo doubling rate maximum 3.5 -> 2039
Algo doubling rate maximum 4 -> 2039
Algo doubling rate maximum 4.5 -> 2039
Algo doubling rate maximum 5 -> 2040
Algo doubling rate maximum 6 -> 2040


In [13]:
print('## Possible Algo Reduction minimum ##')
for m in range(0, 6):
    varset = deepcopy(defaults)
    varset['possible_algo_reduction_'] = possible_algo_reduction_fn(min_reduction=m,
                                                                    max_reduction=5,
                                                                    tai_flop_size=defaults['tai_flop_size_'])
    result = run_tai_model_round(**varset)
    print('Possible algo reduction minimum {} -> {}'.format(m, print_year(result)))

## Possible Algo Reduction minimum ##
Possible algo reduction minimum 0 -> 2043
Possible algo reduction minimum 1 -> 2040
Possible algo reduction minimum 2 -> 2039
Possible algo reduction minimum 3 -> 2039
Possible algo reduction minimum 4 -> 2039
Possible algo reduction minimum 5 -> 2039


In [14]:
print('## Possible Algo Reduction maximum ##')
for m in range(2, 16):
    varset = deepcopy(defaults)
    varset['possible_algo_reduction_'] = possible_algo_reduction_fn(min_reduction=2,
                                                                    max_reduction=m,
                                                                    tai_flop_size=defaults['tai_flop_size_'])
    result = run_tai_model_round(**varset)
    print('Possible algo reduction maximum {} -> {}'.format(m, print_year(result)))

## Possible Algo Reduction maximum ##
Possible algo reduction maximum 2 -> 2039
Possible algo reduction maximum 3 -> 2039
Possible algo reduction maximum 4 -> 2039
Possible algo reduction maximum 5 -> 2039
Possible algo reduction maximum 6 -> 2039
Possible algo reduction maximum 7 -> 2039
Possible algo reduction maximum 8 -> 2039
Possible algo reduction maximum 9 -> 2039
Possible algo reduction maximum 10 -> 2039
Possible algo reduction maximum 11 -> 2039
Possible algo reduction maximum 12 -> 2039
Possible algo reduction maximum 13 -> 2039
Possible algo reduction maximum 14 -> 2039
Possible algo reduction maximum 15 -> 2039


In [15]:
print('## Willingness ramp ##')
for r in range(1, 11):
    varset = deepcopy(defaults)
    varset['willingness_ramp_'] = r / 10
    result = run_tai_model_round(**varset)
    print('Willingness ramp {}x -> {}'.format(round(10 / r, 1), print_year(result)))

## Willingness ramp ##
Willingness ramp 10.0x -> 2035
Willingness ramp 5.0x -> 2036
Willingness ramp 3.3x -> 2037
Willingness ramp 2.5x -> 2037
Willingness ramp 2.0x -> 2038
Willingness ramp 1.7x -> 2038
Willingness ramp 1.4x -> 2038
Willingness ramp 1.2x -> 2038
Willingness ramp 1.1x -> 2039
Willingness ramp 1.0x -> 2039


In [16]:
print('## Willingness spend horizon ##')
for h in range(1, 11):
    varset = deepcopy(defaults)
    varset['willingness_spend_horizon_'] = h
    result = run_tai_model_round(**varset)
    print('Willingness spend horizon {}yrs -> {}'.format(h, print_year(result)))

## Willingness spend horizon ##
Willingness spend horizon 1yrs -> 2039
Willingness spend horizon 2yrs -> 2039
Willingness spend horizon 3yrs -> 2039
Willingness spend horizon 4yrs -> 2039
Willingness spend horizon 5yrs -> 2039
Willingness spend horizon 6yrs -> 2039
Willingness spend horizon 7yrs -> 2039
Willingness spend horizon 8yrs -> 2039
Willingness spend horizon 9yrs -> 2039
Willingness spend horizon 10yrs -> 2039


## Metaculus

In [17]:
get_question(11558)

11558 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates?
6148 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates? (Jan-2022)
Resolved as 320532.1027084993
-
6192 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates? (Jan-2031)
Metaculus -
* Min: 1800
* <Min: 0.0
* Q1: 3632360.778
* Mid: 37316259.269
* Q3: 447769492.694
* >Max: 0.09219999999999995
* Max: 10000000000
-
6517 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates? (Feb-2023)
Resolved as 688639.124123147
-
6559 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates? (Jan-2026)
Metaculus -
* Min: 3640
* <Min: 0.0
* Q1: 700549.46
* Mid: 5889135.777
* Q3: 56111686.0
* >Max: 0.11046999999999996
* Max: 1000000000
-


In [18]:
# https://www.metaculus.com/questions/11558/maximum-compute-used-in-ai-training/
# TODO: Fetch from Metaculus, look side by side
# TODO: Be able to predict back
for y in [2022, 2025, 2030]:
    print('-')
    print('## {} ##'.format(y))
    for i in range(3):
        flop_at_max_ = flop_at_max(initial_gdp=defaults['initial_gdp_'],
                                   gdp_growth=[1.02, 1.025, 1.03][i],
                                   initial_pay=[10*M, 100*M, 1*B][i],
                                   spend_doubling_time=[1, 2, 3][i],
                                   max_gdp_frac=[1/1000, 4/1000, 1/100][i],
                                   initial_flop_per_dollar=10 ** 18,
                                   max_flop_per_dollar=10 ** 24,
                                   flop_halving_rate=[2, 2.5, 3][i],
                                   year=(y - CURRENT_YEAR))

        print('{}: {} max log FLOPs / ~{} petaFLOP/s-days)'.format(['25th', 'mean', '75th'][i],
                                                                   np.round(np.log10(flop_at_max_), 1),
                                                                   log_flop_to_petaflop_sdays(np.log10(flop_at_max_))))


-
## 2022 ##
25th: 24.1 max log FLOPs / ~14466 petaFLOP/s-days)
mean: 25.5 max log FLOPs / ~332188 petaFLOP/s-days)
75th: 26.6 max log FLOPs / ~4579863 petaFLOP/s-days)
-
## 2025 ##
25th: 25.5 max log FLOPs / ~327085 petaFLOP/s-days)
mean: 26.3 max log FLOPs / ~2156562 petaFLOP/s-days)
75th: 27.2 max log FLOPs / ~18275479 petaFLOP/s-days)
-
## 2030 ##
25th: 27.7 max log FLOPs / ~57829880 petaFLOP/s-days)
mean: 27.6 max log FLOPs / ~48506691 petaFLOP/s-days)
75th: 28.2 max log FLOPs / ~182525964 petaFLOP/s-days)


## Anchors sensitivity analysis

In [19]:
for b in range(11, 21):
    anchor = tai_log_flop_needs(brain=b,
                                efficiency=0,
                                transformative_vs_human=0,
                                horizon_length=0,
                                scaling_exponent=1,
                                flops_per_param_per_sec=1,
                                bayes_update=peter_bayes_update_against_low_flop) @ 2
    print('Brain {} -> {} log FLOP'.format(b, round(anchor[0], 1)))

Brain 11 -> 26.9 log FLOP
Brain 12 -> 26.2 log FLOP
Brain 13 -> 28.4 log FLOP
Brain 14 -> 27.8 log FLOP
Brain 15 -> 29.8 log FLOP
Brain 16 -> 31.8 log FLOP
Brain 17 -> 33.8 log FLOP
Brain 18 -> 35.8 log FLOP
Brain 19 -> 37.8 log FLOP
Brain 20 -> 39.8 log FLOP


In [20]:
for e in range(-2, 6):
    anchor = tai_log_flop_needs(brain=15,
                                efficiency=e,
                                transformative_vs_human=0,
                                horizon_length=0,
                                scaling_exponent=1,
                                flops_per_param_per_sec=1,
                                bayes_update=peter_bayes_update_against_low_flop) @ 2
    print('Efficiency {} -> {} log FLOP'.format(e, round(anchor[0], 1)))

Efficiency -2 -> 28.5 log FLOP
Efficiency -1 -> 27.8 log FLOP
Efficiency 0 -> 29.8 log FLOP
Efficiency 1 -> 31.8 log FLOP
Efficiency 2 -> 33.8 log FLOP
Efficiency 3 -> 35.8 log FLOP
Efficiency 4 -> 37.8 log FLOP
Efficiency 5 -> 39.8 log FLOP


In [21]:
for t in range(-3, 4):
    anchor = tai_log_flop_needs(brain=15,
                                efficiency=0,
                                transformative_vs_human=t,
                                horizon_length=0,
                                scaling_exponent=1,
                                flops_per_param_per_sec=1,
                                bayes_update=peter_bayes_update_against_low_flop) @ 2
    print('Transformative vs. human {} -> {} log FLOP'.format(t, round(anchor[0], 1)))

Transformative vs. human -3 -> 26.8 log FLOP
Transformative vs. human -2 -> 27.8 log FLOP
Transformative vs. human -1 -> 28.8 log FLOP
Transformative vs. human 0 -> 29.8 log FLOP
Transformative vs. human 1 -> 30.8 log FLOP
Transformative vs. human 2 -> 31.8 log FLOP
Transformative vs. human 3 -> 32.8 log FLOP


In [22]:
for h in range(0, 10):
    anchor = tai_log_flop_needs(brain=15,
                                efficiency=0,
                                transformative_vs_human=0,
                                horizon_length=h,
                                scaling_exponent=1,
                                flops_per_param_per_sec=1,
                                bayes_update=peter_bayes_update_against_low_flop) @ 2
    print('Horizon length {} -> {} log FLOP'.format(h, round(anchor[0], 1)))

Horizon length 0 -> 29.8 log FLOP
Horizon length 1 -> 30.8 log FLOP
Horizon length 2 -> 31.8 log FLOP
Horizon length 3 -> 32.8 log FLOP
Horizon length 4 -> 33.8 log FLOP
Horizon length 5 -> 34.8 log FLOP
Horizon length 6 -> 35.8 log FLOP
Horizon length 7 -> 36.8 log FLOP
Horizon length 8 -> 37.8 log FLOP
Horizon length 9 -> 38.8 log FLOP


In [23]:
for s in range(5, 16):
    anchor = tai_log_flop_needs(brain=15,
                                efficiency=0,
                                transformative_vs_human=0,
                                horizon_length=0,
                                scaling_exponent=s / 10,
                                flops_per_param_per_sec=1,
                                bayes_update=peter_bayes_update_against_low_flop) @ 2
    print('Scaling exponent {} -> {} log FLOP'.format(s / 10, round(anchor[0], 1)))

Scaling exponent 0.5 -> 28.4 log FLOP
Scaling exponent 0.6 -> 28.7 log FLOP
Scaling exponent 0.7 -> 29.0 log FLOP
Scaling exponent 0.8 -> 29.2 log FLOP
Scaling exponent 0.9 -> 29.5 log FLOP
Scaling exponent 1.0 -> 29.8 log FLOP
Scaling exponent 1.1 -> 30.1 log FLOP
Scaling exponent 1.2 -> 30.4 log FLOP
Scaling exponent 1.3 -> 30.6 log FLOP
Scaling exponent 1.4 -> 30.9 log FLOP
Scaling exponent 1.5 -> 31.2 log FLOP


In [24]:
for fppps in range(10, 21):
    anchor = tai_log_flop_needs(brain=15,
                                efficiency=0,
                                transformative_vs_human=0,
                                horizon_length=0,
                                scaling_exponent=1,
                                flops_per_param_per_sec=fppps,
                                bayes_update=peter_bayes_update_against_low_flop) @ 2
    print('FLOPS per param per sec {} -> {} log FLOP'.format(fppps / 10, round(anchor[0], 1)))

FLOPS per param per sec 1.0 -> 26.1 log FLOP
FLOPS per param per sec 1.1 -> 24.0 log FLOP
FLOPS per param per sec 1.2 -> 27.5 log FLOP
FLOPS per param per sec 1.3 -> 25.2 log FLOP
FLOPS per param per sec 1.4 -> 24.0 log FLOP
FLOPS per param per sec 1.5 -> 24.7 log FLOP
FLOPS per param per sec 1.6 -> 24 log FLOP
FLOPS per param per sec 1.7 -> 24 log FLOP
FLOPS per param per sec 1.8 -> 24 log FLOP
FLOPS per param per sec 1.9 -> 24 log FLOP
FLOPS per param per sec 2.0 -> 24 log FLOP


In [25]:
for bayes_fn in [[None, 'none'],
                 [peter_bayes_update_against_low_flop, 'Peter'],
                 [cotra_bayes_update_against_low_flop, 'Cotra']]:
    anchor = tai_log_flop_needs(brain=10,
                                efficiency=0,
                                transformative_vs_human=0,
                                horizon_length=0,
                                scaling_exponent=1,
                                flops_per_param_per_sec=1,
                                bayes_update=bayes_fn[0])
    if bayes_fn[0] is not None:
        anchor = (anchor @ 2)[0]
    print('Bayes FN {} -> {} log FLOP'.format(bayes_fn[1], round(anchor, 1)))

Bayes FN none -> 19.8 log FLOP
Bayes FN Peter -> 27.7 log FLOP
Bayes FN Cotra -> 25.4 log FLOP


In [26]:
print('File last ran: {}'.format(dt.now()))

File last ran: 2024-04-26 10:23:04.809923
